In [ ]:
%matplotlib inline

import cv2
import matplotlib.pyplot as plt
from IPython import display
import urllib.request

vc = cv2.VideoCapture(0)

img_frame = []
img_frame_bg = []
flag = 0 #一番初めの状態は何もおかれていない0, 物が置かれたら1になる．

if vc.isOpened(): # try to get the first frame
    is_capturing, frame = vc.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
    webcam_preview = plt.imshow(frame)
    img_frame = frame
else:
    is_capturing = False


# 画像の差分を計算
def calc_diff(img1, img2):
    img1_bg = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img1_bg = cv2.GaussianBlur(img1_bg, (21, 21), 0)
    img2_bg = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img2_bg = cv2.GaussianBlur(img2_bg, (21, 21), 0)
    diff = cv2.absdiff(img1_bg, img2_bg)
    diff = cv2.threshold(diff, 100, 255, cv2.THRESH_BINARY)[1]
    diff = cv2.dilate(diff, None, iterations=2)
    return diff


while is_capturing:
    try:    # Lookout for a keyboardInterrupt to stop the script
        is_capturing, frame = vc.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
        diff = calc_diff(img_frame, frame)
        img_frame = frame

        webcam_preview.set_data(frame)
        plt.draw()
        # diffの平均が一定以上なら動いていると判断
        
        if diff.mean() > 10:
            print("動いている")
            plt.title("working")
            if flag == 0:
                flag = 1
            else:
                flag = 0
            
            # 画像を保存
            cv2.imwrite(str(flag)+".jpg", frame)

            # 画像をurllibでpostする
            url = "https://d14b-2405-6587-2340-de00-b3a1-2427-104b-5256.jp.ngrok.io/send/"
            
            # 画像を読み込む
            img = open(str(flag)+".jpg", "rb").read()
            req = urllib.request.Request(url, img, method="POST", headers={"Content-Type": "application/octet-stream"})

            # 画像を送信する
            with urllib.request.urlopen(req) as res:
                body = res.read()
                print(body)
        else:
            plt.title("not working")
            if flag == 0:
                flag = 1
            else:
                flag = 0

        display.clear_output(wait=True)
        display.display(plt.gcf())

        plt.pause(5)    # the pause time is = 1 / framerate

    except KeyboardInterrupt:
        vc.release()
